In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import math
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Load both legitmate & DGA domains

In [ ]:
Domain_data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Super_MLDOMAIN_MCLS2.csv')
Domain_data.drop(["Unnamed: 0"], axis=1, inplace=True)
Domain_data.head()

,domain_name,dga_family,domain,length,entropy,first_Digit_Index,H_entropy,vowel_consonant_ratio,vowel_character_ratio,consonant_character_ratio,level_no,alexa_grams,words_grams
0,odqzesdvd.info,Super_family_2,odqzesdvd,9,2.641604,0,3.324863,0.444444,0.285714,0.642857,2,8.618689,10.200253
1,mcrwnxdzu.info,Super_family_2,mcrwnxdzu,9,3.169925,0,3.664498,0.300000,0.214286,0.714286,2,5.331953,4.442651
2,poxrscvfoy.biz,Super_family_2,poxrscvfoy,10,3.121928,0,3.664498,0.300000,0.214286,0.714286,2,9.758886,11.231941
3,dupzwi.info,Super_family_2,dupzwi,6,2.584963,0,3.277613,0.666667,0.363636,0.545455,2,3.861893,7.928046
4,pgldibr.com,Super_family_2,pgldibr,7,2.807355,0,3.459432,0.250000,0.181818,0.727273,2,12.146719,20.785039


In [ ]:
Domain_data.domain=Domain_data.domain.astype(str)

In [ ]:
from sklearn.preprocessing import LabelEncoder
X = Domain_data.domain
Y = Domain_data['dga_family']

In [ ]:
#from sklearn.preprocessing import LabelBinarizer
#Y = LabelBinarizer().fit_transform(Y)

In [ ]:
le=LabelEncoder()
Y = le.fit_transform(Y)

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.models import Sequential

#max_words = num_words
max_len = 50

# Tokenizer
tok = Tokenizer(num_words=None, char_level=True, oov_token='UNK')
tok.fit_on_texts(X)
sequences = tok.texts_to_sequences(X)
sequences_matrix = sequence.pad_sequences(sequences,padding='post',maxlen=max_len)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(sequences_matrix, Y, test_size=0.20,random_state=42)

In [ ]:
from imblearn.over_sampling import RandomOverSampler
strategy = {2:166131,3:166131,4:249197, 5:166131} 
ovrsmt = RandomOverSampler(sampling_strategy=strategy)

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
udrsmt = RandomUnderSampler(sampling_strategy={0:332262,1:581459})

In [ ]:
from imblearn.pipeline import Pipeline
steps = [('o', ovrsmt), ('u', udrsmt)]
pipeline = Pipeline(steps=steps)

In [ ]:
X_train, y_train = pipeline.fit_resample(X_train, y_train)

In [ ]:
import pickle
# save the model to disk
filename = '/content/drive/My Drive/Colab Notebooks/sMX_train_sm.csv'
pickle.dump(X_train, open(filename, 'wb'))

In [ ]:
filename = '/content/drive/My Drive/Colab Notebooks/sMX_test_sm.csv'
pickle.dump(X_test, open(filename, 'wb'))


In [ ]:
filename = '/content/drive/My Drive/Colab Notebooks/sMy_train_sm.csv'
pickle.dump(y_train, open(filename, 'wb'))

In [ ]:
filename = '/content/drive/My Drive/Colab Notebooks/sMy_test_sm.csv'
pickle.dump(y_test, open(filename, 'wb'))

## Load Model and extract feature from an intermidiate layer

In [ ]:
import pickle
# load the model from disk
filename = '/content/drive/My Drive/Colab Notebooks/SMCLS_lstmmodel64,1000x100.sav'
loaded_model = pickle.load(open(filename, 'rb'))

In [ ]:
import pickle
filename = '/content/drive/My Drive/Colab Notebooks/SMX_train_sm.csv'
X_train = pickle.load(open(filename, 'rb'))

In [ ]:
filename = '/content/drive/My Drive/Colab Notebooks/SMy_train_sm.csv'
y_train = pickle.load(open(filename, 'rb'))

In [ ]:
filename = '/content/drive/My Drive/Colab Notebooks/SMX_test_sm.csv'
X_test = pickle.load(open(filename, 'rb'))

In [ ]:
filename = '/content/drive/My Drive/Colab Notebooks/SMy_test_sm.csv'
y_test = pickle.load(open(filename, 'rb'))

In [ ]:
from tensorflow import keras
layer_name = 'flatten_2'
intermediate_layer_model = keras.Model(inputs=loaded_model.input,
                                 outputs=loaded_model.get_layer(layer_name).output)
intermediate_output = intermediate_layer_model.predict(X_train)

### Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
import time
start = time.time()
DTmodel = DecisionTreeClassifier().fit(intermediate_output, y_train)
end = time.time()
print("It took {} milliseconds to train DT with feature from multiclass LSTM intermidiate layer.\n".format(end-start))

It took 543.9129936695099 milliseconds to train DT with feature from multiclass LSTM intermidiate layer.



In [ ]:
from sklearn.metrics import precision_score, accuracy_score,classification_report
start = time.time()
DTpred = DTmodel.predict(intermediate_layer_model.predict(X_test))
end = time.time()
print(classification_report(y_test, DTpred))
print("It took {} milliseconds to test DT with feature from multiclass LSTM intermidiate layer.\n".format(end-start))

              precision    recall  f1-score   support

           0       0.98      0.98      0.98    122348
           1       0.99      0.99      0.99    250598
           2       0.99      0.99      0.99     40114
           3       0.95      0.95      0.95       257
           4       0.87      0.79      0.83      1983
           5       1.00      1.00      1.00        28

    accuracy                           0.98    415328
   macro avg       0.96      0.95      0.96    415328
weighted avg       0.98      0.98      0.98    415328

It took 99.53607153892517 milliseconds to test DT with feature from multiclass LSTM intermidiate layer.



In [ ]:
print('accuracy on the train set: ', accuracy_score(DTmodel.predict(intermediate_output), y_train))
print('accuracy on the test set: ', accuracy_score(DTpred, y_test))

accuracy on the train set:  0.9999969903287224
accuracy on the test set:  0.9849347985206872


In [ ]:
from sklearn import metrics
import matplotlib.pyplot as plt

etpprd = DTmodel.predict_proba(intermediate_layer_model.predict(X_test))

In [ ]:
auc = metrics.roc_auc_score(y_test, etpprd, multi_class='ovo', average='weighted')
print(auc)

0.9763042861899536


In [ ]:
import pickle
# save the model to disk
filename = '/content/drive/My Drive/Colab Notebooks/sMLSTM+DTmodel.sav'
pickle.dump(DTmodel, open(filename, 'wb'))

### Random Forest Classifier

In [ ]:
import time
from sklearn.ensemble import RandomForestClassifier
start = time.time()
RFmodel = RandomForestClassifier().fit(intermediate_output, y_train)
end = time.time()
print("It took {} milliseconds to train RF with feature from multiclass LSTM intermidiate layer.\n".format(end-start))

It took 3957.920889854431 milliseconds to train RF with feature from multiclass LSTM intermidiate layer.



In [ ]:
from sklearn.metrics import precision_score, accuracy_score,classification_report
start = time.time()
RFpred = RFmodel.predict(intermediate_layer_model.predict(X_test))
end = time.time()
print("It took {} milliseconds to test LR with feature from multiclass LSTM intermidiate layer.\n".format(end-start))
print(classification_report(y_test, RFpred))

It took 109.30394649505615 milliseconds to test LR with feature from multiclass LSTM intermidiate layer.

              precision    recall  f1-score   support

           0       0.98      0.99      0.98    122348
           1       0.99      0.99      0.99    250598
           2       0.99      0.99      0.99     40114
           3       0.95      0.96      0.95       257
           4       0.86      0.81      0.84      1983
           5       1.00      1.00      1.00        28

    accuracy                           0.99    415328
   macro avg       0.96      0.96      0.96    415328
weighted avg       0.99      0.99      0.99    415328



In [ ]:
print('accuracy on the train set: ', accuracy_score(RFmodel.predict(intermediate_output), y_train))
print('accuracy on the test set: ', accuracy_score(RFpred, y_test))

accuracy on the train set:  0.9999951845259557
accuracy on the test set:  0.9890664727636952


In [ ]:
from sklearn import metrics
import matplotlib.pyplot as plt

etpprd = RFmodel.predict_proba(intermediate_layer_model.predict(X_test))

In [ ]:
auc = metrics.roc_auc_score(y_test, etpprd, multi_class='ovo', average='weighted')
print(auc)

0.9896814446267379


In [ ]:
import pickle
# save the model to disk
filename = '/content/drive/My Drive/Colab Notebooks/sMLSTM+RFmodel.sav'
pickle.dump(RFmodel, open(filename, 'wb'))

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
precision=precision_score(RFmodel.predict(intermediate_layer_model.predict(X_test)),y_test)
recall=recall_score(RFmodel.predict(intermediate_layer_model.predict(X_test)),y_test)
f1=f1_score(RFmodel.predict(intermediate_layer_model.predict(X_test)),y_test)
accuracy=accuracy_score(RFmodel.predict(intermediate_layer_model.predict(X_test)),y_test)
print(accuracy,precision,recall,f1)

### Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
import time
start = time.time()
Lreg = LogisticRegression().fit(intermediate_output, y_train)
end = time.time()
print("It took {} milliseconds to train LR with feature from multiclass LSTM intermidiate layer.\n".format(end-start))

It took 125.89345169067383 milliseconds to train LR with feature from multiclass LSTM intermidiate layer.



/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [ ]:
from sklearn.metrics import precision_score, accuracy_score,classification_report
start = time.time()
Lregy_pred = Lreg.predict(intermediate_layer_model.predict(X_test))
end = time.time()
print("It took {} milliseconds to test LR with feature from multiclass LSTM intermidiate layer.\n".format(end-start))
print(classification_report(y_test, Lregy_pred))

It took 96.48069739341736 milliseconds to test LR with feature from multiclass LSTM intermidiate layer.

              precision    recall  f1-score   support

           0       0.98      0.99      0.98    122348
           1       0.99      0.99      0.99    250598
           2       0.99      0.99      0.99     40114
           3       0.92      0.98      0.95       257
           4       0.80      0.84      0.82      1983
           5       0.97      1.00      0.98        28

    accuracy                           0.99    415328
   macro avg       0.94      0.96      0.95    415328
weighted avg       0.99      0.99      0.99    415328



In [ ]:
print('accuracy on the train set: ', accuracy_score(Lreg.predict(intermediate_output), y_train))
print('accuracy on the test set: ', accuracy_score(Lregy_pred, y_test))

accuracy on the train set:  0.9948594814577162
accuracy on the test set:  0.9885415864088143


In [ ]:
from sklearn import metrics
import matplotlib.pyplot as plt

etpprd = Lreg.predict_proba(intermediate_layer_model.predict(X_test))

In [ ]:
auc = metrics.roc_auc_score(y_test, etpprd, multi_class='ovo', average='weighted')
print(auc)

0.9966374652741955


In [ ]:
import pickle
# save the model to disk
filename = '/content/drive/My Drive/Colab Notebooks/sMLSTM+LRmodel.sav'
pickle.dump(Lreg, open(filename, 'wb'))

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
precision=precision_score(Lreg.predict(intermediate_layer_model.predict(X_test)),y_test)
recall=recall_score(Lreg.predict(intermediate_layer_model.predict(X_test)),y_test)
f1=f1_score(Lreg.predict(intermediate_layer_model.predict(X_test)),y_test)
accuracy=accuracy_score(Lreg.predict(intermediate_layer_model.predict(X_test)),y_test)
print(accuracy,precision,recall,f1

### Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB
start = time.time()
gnb = GaussianNB().fit(intermediate_output, y_train)
end = time.time()
print("It took {} milliseconds to train NB with feature from multiclass LSTM intermidiate layer.\n".format(end-start))

It took 1.800511121749878 milliseconds to train NB with feature from multiclass LSTM intermidiate layer.



In [ ]:
from sklearn.metrics import precision_score, accuracy_score,classification_report
start = time.time()
gnb_pred = gnb.predict(intermediate_layer_model.predict(X_test))
end = time.time()
print("It took {} milliseconds to test NB with feature from multiclass LSTM intermidiate layer.\n".format(end-start))
print(classification_report(y_test, gnb_pred))

It took 143.36213326454163 milliseconds to test NB with feature from multiclass LSTM intermidiate layer.

              precision    recall  f1-score   support

           0       0.95      0.99      0.97    122348
           1       1.00      0.97      0.98    250598
           2       0.99      0.99      0.99     40114
           3       0.68      0.98      0.80       257
           4       0.68      0.84      0.75      1983
           5       1.00      1.00      1.00        28

    accuracy                           0.98    415328
   macro avg       0.88      0.96      0.92    415328
weighted avg       0.98      0.98      0.98    415328



In [ ]:
print('accuracy on the train set: ', accuracy_score(gnb.predict(intermediate_output), y_train))
print('accuracy on the test set: ', accuracy_score(gnb_pred, y_test))

accuracy on the train set:  0.9870481806236159
accuracy on the test set:  0.9780703444024963


In [ ]:
from sklearn import metrics
import matplotlib.pyplot as plt

etpprd = gnb.predict_proba(intermediate_layer_model.predict(X_test))

In [ ]:
auc = metrics.roc_auc_score(y_test, etpprd, multi_class='ovo', average='weighted')
print(auc)

0.9922636679509432


In [ ]:
import pickle
# save the model to disk
filename = '/content/drive/My Drive/Colab Notebooks/sMLSTM+NBmodel.sav'
pickle.dump(gnb, open(filename, 'wb'))

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
precision=precision_score(gnb.predict(intermediate_layer_model.predict(X_test)),y_test)
recall=recall_score(gnb.predict(intermediate_layer_model.predict(X_test)),y_test)
f1=f1_score(gnb.predict(intermediate_layer_model.predict(X_test)),y_test)
accuracy=accuracy_score(gnb.predict(intermediate_layer_model.predict(X_test)),y_test)
print(accuracy,precision,recall,f1

### Adaboost Classifier

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
import time
start = time.time()
ABmodel = AdaBoostClassifier(DecisionTreeClassifier()).fit(intermediate_output, y_train)
end = time.time()
print("It took {} milliseconds to train AB with feature from multiclass LSTM intermidiate layer.\n".format(end-start))

It took 17002.392912864685 milliseconds to train AB with feature from multiclass LSTM intermidiate layer.



In [ ]:
from sklearn.metrics import precision_score, accuracy_score,classification_report
start = time.time()
ABpred = ABmodel.predict(intermediate_layer_model.predict(X_test))
end = time.time()
print(classification_report(y_test, ABpred))
print("It took {} milliseconds to test AB with feature from multiclass LSTM intermidiate layer.\n".format(end-start))

              precision    recall  f1-score   support

           0       0.98      0.99      0.98    122348
           1       0.99      0.99      0.99    250598
           2       0.99      0.99      0.99     40114
           3       0.95      0.96      0.95       257
           4       0.87      0.81      0.84      1983
           5       1.00      1.00      1.00        28

    accuracy                           0.99    415328
   macro avg       0.96      0.96      0.96    415328
weighted avg       0.99      0.99      0.99    415328

It took 117.32239985466003 milliseconds to test AB with feature from multiclass LSTM intermidiate layer.



In [ ]:
print('accuracy on the train set: ', accuracy_score(ABmodel.predict(intermediate_output), y_train))
print('accuracy on the test set: ', accuracy_score(ABpred, y_test))

accuracy on the train set:  0.9999969903287224
accuracy on the test set:  0.9888401456198475


In [ ]:
from sklearn import metrics
import matplotlib.pyplot as plt

etpprd = ABmodel.predict_proba(intermediate_layer_model.predict(X_test))

In [ ]:
auc = metrics.roc_auc_score(y_test, etpprd, multi_class='ovo', average='weighted')
print(auc)

0.9892365905198801


In [ ]:
import pickle
# save the model to disk
filename = '/content/drive/My Drive/Colab Notebooks/sMLSTM+ABmodel.sav'
pickle.dump(ABmodel, open(filename, 'wb'))

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
precision=precision_score(ABmodel.predict(intermediate_layer_model.predict(X_test)),y_test)
recall=recall_score(gnb.predict(intermediate_layer_model.predict(X_test)),y_test)
f1=f1_score(gnb.predict(intermediate_layer_model.predict(X_test)),y_test)
accuracy=accuracy_score(gnb.predict(intermediate_layer_model.predict(X_test)),y_test)
print(accuracy,precision,recall,f1

### XGBoost Classifier

In [ ]:
import time
from xgboost import XGBClassifier
start = time.time()
XGBmodel = XGBClassifier().fit(intermediate_output, y_train)
end = time.time()
print("It took {} milliseconds to train XGB with feature from multiclass LSTM intermidiate layer.\n".format(end-start))

It took 5280.543651103973 milliseconds to train XGB with feature from multiclass LSTM intermidiate layer.



In [ ]:
from sklearn.metrics import precision_score, accuracy_score,classification_report
start = time.time()
xgbpred = XGBmodel.predict(intermediate_layer_model.predict(X_test))
end = time.time()
print("It took {} milliseconds to test XGB with feature from multiclass LSTM intermidiate layer.\n".format(end-start))
print(classification_report(y_test, xgbpred))

It took 119.38405537605286 milliseconds to test XGB with feature from multiclass LSTM intermidiate layer.

              precision    recall  f1-score   support

           0       0.98      0.98      0.98    122348
           1       0.99      0.99      0.99    250598
           2       0.99      0.99      0.99     40114
           3       0.93      0.98      0.95       257
           4       0.80      0.84      0.82      1983
           5       1.00      1.00      1.00        28

    accuracy                           0.99    415328
   macro avg       0.95      0.96      0.96    415328
weighted avg       0.99      0.99      0.99    415328



In [ ]:
print('accuracy on the train set: ', accuracy_score(XGBmodel.predict(intermediate_output), y_train))
print('accuracy on the test set: ', accuracy_score(xgbpred, y_test))

accuracy on the train set:  0.9947565507000194
accuracy on the test set:  0.9883971222744433


In [ ]:
from sklearn import metrics
import matplotlib.pyplot as plt

etpprd = XGBmodel.predict_proba(intermediate_layer_model.predict(X_test))

In [ ]:
auc = metrics.roc_auc_score(y_test, etpprd, multi_class='ovo', average='weighted')
print(auc)

0.9940980301256312


In [ ]:
import pickle
# save the model to disk
filename = '/content/drive/My Drive/Colab Notebooks/SMLSTM+XGBmodel.sav'
pickle.dump(XGBmodel, open(filename, 'wb'))

## CNN

In [ ]:
import pickle
# load the model from disk
filename = '/content/drive/My Drive/Colab Notebooks/SMcnnmodel_2x64x5,1000x100.sav'
loaded_model = pickle.load(open(filename, 'rb'))

In [ ]:
from tensorflow import keras
layer_name = 'flatten_2'
intermediate_layer_model = keras.Model(inputs=loaded_model.input,
                                 outputs=loaded_model.get_layer(layer_name).output)
intermediate_output2 = intermediate_layer_model.predict(X_train)

## Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
import time
start = time.time()
DTmodel = DecisionTreeClassifier().fit(intermediate_output2, y_train)
end = time.time()
print("It took {} milliseconds to train DT with feature from multiclass CNN intermidiate layer.\n".format(end-start))

It took 458.1474094390869 milliseconds to train DT with feature from multiclass CNN intermidiate layer.



In [ ]:
from sklearn.metrics import precision_score, accuracy_score,classification_report
start = time.time()
DTpred = DTmodel.predict(intermediate_layer_model.predict(X_test))
end = time.time()
print(classification_report(y_test, DTpred))
print("It took {} milliseconds to test DT with feature from multiclass CNN intermidiate layer.\n".format(end-start))

              precision    recall  f1-score   support

           0       0.95      0.95      0.95    122348
           1       0.98      0.97      0.98    250598
           2       0.97      0.98      0.97     40114
           3       0.84      0.80      0.82       257
           4       0.55      0.58      0.57      1983
           5       1.00      1.00      1.00        28

    accuracy                           0.96    415328
   macro avg       0.88      0.88      0.88    415328
weighted avg       0.96      0.96      0.96    415328

It took 82.1825852394104 milliseconds to test DT with feature from multiclass CNN intermidiate layer.



In [ ]:
print('accuracy on the train set: ', accuracy_score(DTmodel.predict(intermediate_output2), y_train))
print('accuracy on the test set: ', accuracy_score(DTpred, y_test))

accuracy on the train set:  0.9999969903287224
accuracy on the test set:  0.9646953732953232


In [ ]:
from sklearn import metrics
import matplotlib.pyplot as plt

etpprd = DTmodel.predict_proba(intermediate_layer_model.predict(X_test))

In [ ]:
#fpr,tpr, _ = metrics.roc_curve(y_test,etpprd)
#auc = metrics.roc_auc_score(y_test,etpprd)
#roc auc score
auc = metrics.roc_auc_score(y_test, etpprd, multi_class='ovo', average='weighted')

In [ ]:
print(auc)

0.9433628455254537


In [ ]:
import pickle
# save the model to disk
filename = '/content/drive/My Drive/Colab Notebooks/sMCNN+DTmodel.sav'
pickle.dump(DTmodel, open(filename, 'wb'))

## Naive Bayes

In [ ]:
import time
from sklearn.naive_bayes import GaussianNB
start = time.time()
gnb = GaussianNB().fit(intermediate_output2, y_train)
end = time.time()
print("It took {} milliseconds to train NB with feature from multiclass CNN intermidiate layer.\n".format(end-start))

It took 1.5647809505462646 milliseconds to train NB with feature from multiclass CNN intermidiate layer.



In [ ]:
from sklearn.metrics import precision_score, accuracy_score,classification_report
start = time.time()
gnb_pred = gnb.predict(intermediate_layer_model.predict(X_test))
end = time.time()
print("It took {} milliseconds to test NB with feature from multiclass CNN intermidiate layer.\n".format(end-start))
print(classification_report(y_test, gnb_pred))

It took 70.14196825027466 milliseconds to test NB with feature from multiclass CNN intermidiate layer.

              precision    recall  f1-score   support

           0       0.88      0.83      0.86    122348
           1       0.99      0.94      0.96    250598
           2       0.97      0.99      0.98     40114
           3       0.15      0.93      0.25       257
           4       0.08      0.84      0.15      1983
           5       0.88      1.00      0.93        28

    accuracy                           0.91    415328
   macro avg       0.66      0.92      0.69    415328
weighted avg       0.95      0.91      0.93    415328



In [ ]:
print('accuracy on the train set: ', accuracy_score(gnb.predict(intermediate_output2), y_train))
print('accuracy on the test set: ', accuracy_score(gnb_pred, y_test))

accuracy on the train set:  0.920665667054513
accuracy on the test set:  0.9101023769165575


In [ ]:
from sklearn import metrics
import matplotlib.pyplot as plt

etpprd = gnb.predict_proba(intermediate_layer_model.predict(X_test))

In [ ]:
#fpr,tpr, _ = metrics.roc_curve(y_test,etpprd)
#auc = metrics.roc_auc_score(y_test,etpprd)
#roc auc score
auc = metrics.roc_auc_score(y_test, etpprd, multi_class='ovo', average='weighted')

In [ ]:
print(auc)

0.9882201217610431


In [ ]:
import pickle
# save the model to disk
filename = '/content/drive/My Drive/Colab Notebooks/sMCNN+NBmodel.sav'
pickle.dump(gnb, open(filename, 'wb'))

## Logistic Regression

In [ ]:
import time
from sklearn.linear_model import LogisticRegression
start = time.time()
Lreg = LogisticRegression().fit(intermediate_output2, y_train)
end = time.time()
print("It took {} milliseconds to train LR with feature from multiclass CNN intermidiate layer.\n".format(end-start))

It took 137.6460464000702 milliseconds to train LR with feature from multiclass CNN intermidiate layer.



/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [ ]:
from sklearn.metrics import precision_score, accuracy_score,classification_report
start = time.time()
Lregy_pred = Lreg.predict(intermediate_layer_model.predict(X_test))
end = time.time()
print("It took {} milliseconds to test LR with feature from multiclass CNN intermidiate layer.\n".format(end-start))
print(classification_report(y_test, Lregy_pred))

It took 62.5492639541626 milliseconds to test LR with feature from multiclass CNN intermidiate layer.

              precision    recall  f1-score   support

           0       0.97      0.97      0.97    122348
           1       0.99      0.98      0.99    250598
           2       0.98      0.98      0.98     40114
           3       0.70      0.95      0.80       257
           4       0.65      0.84      0.74      1983
           5       0.90      1.00      0.95        28

    accuracy                           0.98    415328
   macro avg       0.87      0.95      0.90    415328
weighted avg       0.98      0.98      0.98    415328



In [ ]:
print('accuracy on the train set: ', accuracy_score(Lreg.predict(intermediate_output2), y_train))
print('accuracy on the test set: ', accuracy_score(Lregy_pred, y_test))

accuracy on the train set:  0.9896822449258447
accuracy on the test set:  0.9799869019184837


In [ ]:
from sklearn import metrics
import matplotlib.pyplot as plt

etpprd = Lreg.predict_proba(intermediate_layer_model.predict(X_test))

In [ ]:
#fpr,tpr, _ = metrics.roc_curve(y_test,etpprd)
#auc = metrics.roc_auc_score(y_test,etpprd)
#roc auc score
auc = metrics.roc_auc_score(y_test, etpprd, multi_class='ovo', average='weighted')

In [ ]:
print(auc)

0.996494937279581


In [ ]:
import pickle
# save the model to disk
filename = '/content/drive/My Drive/Colab Notebooks/sMCNN+LRmodel.sav'
pickle.dump(Lreg, open(filename, 'wb'))

## Random Forest Classifier

In [ ]:
import time
from sklearn.ensemble import RandomForestClassifier
start = time.time()
RFmodel = RandomForestClassifier().fit(intermediate_output2, y_train)
end = time.time()
print("It took {} milliseconds to train RF with feature from multiclass CNN intermidiate layer.\n".format(end-start))

It took 3190.4395730495453 milliseconds to train RF with feature from multiclass CNN intermidiate layer.



In [ ]:
from sklearn.metrics import precision_score, accuracy_score,classification_report
start = time.time()
RFpred = RFmodel.predict(intermediate_layer_model.predict(X_test))
end = time.time()
print("It took {} milliseconds to test LR with feature from multiclass CNN intermidiate layer.\n".format(end-start))
print(classification_report(y_test, RFpred))

It took 72.34440016746521 milliseconds to test LR with feature from multiclass CNN intermidiate layer.

              precision    recall  f1-score   support

           0       0.96      0.99      0.97    122348
           1       0.99      0.98      0.99    250598
           2       0.99      0.99      0.99     40114
           3       0.91      0.88      0.90       257
           4       0.98      0.56      0.71      1983
           5       1.00      1.00      1.00        28

    accuracy                           0.98    415328
   macro avg       0.97      0.90      0.93    415328
weighted avg       0.98      0.98      0.98    415328



In [ ]:
print('accuracy on the train set: ', accuracy_score(RFmodel.predict(intermediate_output2), y_train))
print('accuracy on the test set: ', accuracy_score(RFpred, y_test))

accuracy on the train set:  0.9999957864602113
accuracy on the test set:  0.9828545149857462


In [ ]:
from sklearn import metrics
import matplotlib.pyplot as plt

etpprd = RFmodel.predict_proba(intermediate_layer_model.predict(X_test))

In [ ]:
#fpr,tpr, _ = metrics.roc_curve(y_test,etpprd)
#auc = metrics.roc_auc_score(y_test,etpprd)
#roc auc score
auc = metrics.roc_auc_score(y_test, etpprd, multi_class='ovo', average='weighted')

In [ ]:
print(auc)

0.9962406191675333


In [ ]:
import pickle
# save the model to disk
filename = '/content/drive/My Drive/Colab Notebooks/sMCNN+RFmodel.sav'
pickle.dump(RFmodel, open(filename, 'wb'))

## Adaboost Classifier

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
import time
start = time.time()
ABmodel = AdaBoostClassifier(DecisionTreeClassifier()).fit(intermediate_output2 , y_train)
end = time.time()
print("It took {} milliseconds to train AB with feature from multiclass CNN intermidiate layer.\n".format(end-start))

It took 14440.517945289612 milliseconds to train AB with feature from multiclass CNN intermidiate layer.



In [ ]:
from sklearn.metrics import precision_score, accuracy_score,classification_report
start = time.time()
ABpred = ABmodel.predict(intermediate_layer_model.predict(X_test))
end = time.time()
print(classification_report(y_test, ABpred))
print("It took {} milliseconds to test AB with feature from multiclass CNN intermidiate layer.\n".format(end-start))

              precision    recall  f1-score   support

           0       0.96      0.99      0.98    122348
           1       0.99      0.98      0.99    250598
           2       0.99      0.99      0.99     40114
           3       0.95      0.87      0.91       257
           4       0.99      0.54      0.69      1983
           5       1.00      1.00      1.00        28

    accuracy                           0.98    415328
   macro avg       0.98      0.89      0.93    415328
weighted avg       0.98      0.98      0.98    415328

It took 64.48911619186401 milliseconds to test AB with feature from multiclass CNN intermidiate layer.



In [ ]:
print('accuracy on the train set: ', accuracy_score(ABmodel.predict(intermediate_output2), y_train))
print('accuracy on the test set: ', accuracy_score(ABpred, y_test))

accuracy on the train set:  0.9999975922629779
accuracy on the test set:  0.9835431273595808


In [ ]:
from sklearn import metrics
import matplotlib.pyplot as plt

etpprd = ABmodel.predict_proba(intermediate_layer_model.predict(X_test))

In [ ]:
#fpr,tpr, _ = metrics.roc_curve(y_test,etpprd)
#auc = metrics.roc_auc_score(y_test,etpprd)
#roc auc score
auc = metrics.roc_auc_score(y_test, etpprd, multi_class='ovo', average='weighted')

In [ ]:
print(auc)

0.9957760105910728


In [ ]:
import pickle
# save the model to disk
filename = '/content/drive/My Drive/Colab Notebooks/sMCNN+ABmodel.sav'
pickle.dump(ABmodel, open(filename, 'wb'))